## Introduction to Linear and Discrete Optimization (ADM I): Programming Exercise 1

Technische Universität Berlin, Straße des 17. Juni 135, 10623 Berlin, Deutschland

---

### Homework Group 7

**Mikhail Farber, Tim Rüttinger and Allan A. Zea**

*Disclaimer: The use of this material without permission of the authors is strictly prohibited.*

---

The code in this cell installs the required software:

In [1]:
!pip install picos
!pip install gurobipy
import picos
import numpy as np

     |████████████████████████████████| 332 kB 38.1 MB/s 
  Created wheel for picos: filename=PICOS-2.3.1-py3-none-any.whl size=469700 sha256=75c9758359d00df329617e765bbf0f917bd472444a58a6fc440cc5b405b5a504
  Stored in directory: /root/.cache/pip/wheels/96/9d/7d/dbb86007eff747cef942f62276888ccba93ad1f006c216476b
Successfully built picos
     |████████████████████████████████| 11.5 MB 25.2 MB/s 


Consider the following problem. 

**Given**: a finite set $N = \{1,\ldots , n\}$, distances $d_{ij} \geq 0$ for all $i, j \in N$ with $d_{ii} = 0$ for all $i\in N$ ,
and an integer $k \in \{1, \ldots , n\}$.

**Task**: choose a subset $C \subseteq N$ with $|C| \leq k$ such that $\max_{i\in N} \min_{j\in C} d_{ij}$ is minimized.

Generate some unique data for the problem using the following cell, by replacing `SEED` by the number of your homework group. Then, formulate the above optimization problem as an IP and solve the problem with the method of your choice: you can either use the  PICOS/gurobi framwork that is pre-installed in this notebook, or download the data as a csv file and solve the problem with your favorite solver or interface (ZIMPL/SCIP, cvxpy with any solver, gurobipy interface to gurobi, ...).

**Hint**
The task can be reformulated as follows: Choose a subset $C \subseteq N$ and a mapping $a \colon N \to C$ such that $\max_{i \in N} d_{i,a(i)}$ is minimized.

Return the solution via ISIS by outputing the solution in the following form: if the optimal max-min distance is 52, and the subset C is {1,4,5}, write
``D = 52; C = {1, 4, 5}`` in the text field on ISIS.

In [10]:
#seed the pseudo-random number generator with your group number
np.random.seed(7)

#generate the data
n, k = 20, 5
d = np.random.randint(1, 101, (n,n)) # generate random integer matrix
d = d + d.T                          # make data symmetric
np.fill_diagonal(d, 0)              # zero-out the diagonal entries

#uncomment the following block to save the data as a .json file that you can
#use with your favorite software
"""
from google.colab import files
import json

data = {'n':n,
        'k':k,
        'd':{i: {j: int(d[i,j]) for j in range(n)}
             for i in range(n)}
        }
with open('ADM1_prog_ex1_data.json','w') as fp:
  json.dump(data, fp, indent=4)

files.download('ADM1_prog_ex1_data.json')
"""

"\nfrom google.colab import files\nimport json\n\ndata = {'n':n,\n        'k':k,\n        'd':{i: {j: int(d[i,j]) for j in range(n)}\n             for i in range(n)}\n        }\nwith open('ADM1_prog_ex1_data.json','w') as fp:\n  json.dump(data, fp, indent=4)\n\nfiles.download('ADM1_prog_ex1_data.json')\n"

---

## Our approach:

To implement the integer linear program (IP) that solves this optimization problem, we first define two decision variables: $\tilde{C}=[C_1,\dots,C_n]$ and $A=({a_{ij}})_{1\leq i,j\leq n}$. The vector $\tilde{C}$ is just a representation of the chosen subset $C\subseteq N$, where $C_j=1$ if index $j\in N$ belongs to $C$ and $C_j=0$ otherwise. We allow a small abuse of notation for the code section and let $C=\tilde{C}$. The matrix $A$ helps us decide which distances one should take into account for the calculation of the given maximum; its entries $a_{ij}$ are given by $a_{ij}=1$ if $d_{ij}=\min_{j'\in C} d_{ij'}$ and $a_{ij}=0$ otherwise. The action of the mapping $a:N\rightarrow C$ can therefore be summarized as $d_{i,a(i)}=\sum_{j=1}^n d_{ij}a_{ij}$ for all $i\in N$, which allows us to rephrase our objective function as $\max_{i\in N} d_{i,a(i)}=\max_{i\in N}\sum_{j=1}^n d_{ij}a_{ij}$. Using the results on the optimization of piecewise convex objective functions (Chapter 2), we obtain the following IP with the respective constraints:

$$
\text{minimize}\quad z\qquad\qquad\qquad\qquad\quad\\
\text{subject to}\quad z\geq\sum_{j=1}^n d_{ij} a_{ij},\quad\forall i\in N\\
\qquad\sum_{j=1}^n C_{j}\leq k,\\
\qquad\qquad\qquad\sum_{j=1}^n a_{ij} = 1,\quad \forall i\in N\\
\qquad\quad\quad\sum_{i=1}^n a_{ij} \leq C_j\cdot k,\quad \forall j\in N\\
\qquad\qquad\qquad\qquad\quad a_{ij}\in\{0,1\},\quad \forall i\in N,~j\in N\\
\qquad\qquad\qquad C_{j}\in\{0,1\},\quad \forall j\in N
$$


We can now use the PICOS library to study this formulation.

In [15]:
P = picos.Problem()

# Variables
z = picos.IntegerVariable("z", lower = 0)
a = picos.BinaryVariable("a", (n, n))
C = picos.BinaryVariable("C", n)

# Objective function
P.set_objective("min", z)
print(P)

# Constraints
for i in range(n):
  P += (sum([a[i,j] for j in range(n)]) == 1)
  
P += (sum([C[j] for j in range(n)]) <= k)

for j in range(n):
  P += (sum([a[i,j] for i in range(n)]) <= C[j] * k)

for i in range(n):
  P += (z >= sum([d[i,j] * a[i,j] for j in range(n)]))

P.solve(verbosity = 1, solver = 'gurobi')

Integer Linear Program
  minimize z
  over
    1×1 integer variable z (bounded below)
            PICOS 2.3.1            
Problem type: Integer Linear Program.
Searching a solution strategy for Gurobi.
Solution strategy:
  1. ExtraOptions
  2. GurobiSolver
Applying ExtraOptions.
Building a Gurobi problem instance.
Starting solution search.
Set parameter FeasibilityTol to value 1e-08
Set parameter OptimalityTol to value 1e-08
Set parameter BarQCPConvTol to value 1e-10
Set parameter MIPGapAbs to value 1e-06
-----------------------------------
         Gurobi Optimizer          
-----------------------------------
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 61 rows, 421 columns and 1240 nonzeros
Model fingerprint: 0x8496d4aa
Variable types: 0 continuous, 421 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds r

<feasible primal solution (claimed optimal) from gurobi>

You can print the solution in the next cell:

In [16]:
print(f'Objective value: {P.value}')
print(f'Optimal solution')
print(f'z     = {z.np}')
print(f'a  = {a.np}')
print(f'C    = {C.np}')

Objective value: 56.0
Optimal solution
z     = 56.0
a  = [[ 1.  0. -0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  -0. -0.]
 [ 0.  1. -0. -0. -0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0. -0.  0.  0.
   0.  0.]
 [-0. -0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  1. -0.
   0.  0.]
 [ 0.  1.  0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  1.  0.  0. -0. -0. -0.  0.  0.  0.  0.  0. -0.  0. -0.  0.  0. -0.
  -0. -0.]
 [ 1.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  1.  0. -0. -0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0.  0.  0.  0. -0.  0.  0.  0.  1.  0.
   0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0. -0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  1.  0. -0.  0.  0.
   0.  0.]
 [ 0.  1.  0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0

From here, we see that the global optimum is $z^{*}=56$ and that $C=\{1,2,5,14,17\}$ is a candidate index subset that solves the problem.

---

## Alternative approach: Interpretation as the $k$-center problem

Upon noticing that the problem presented above is equivalent to the so-called $k$-center problem -- in which one tries to select a given number $k$ of facilities from possible $n$ points in a graph, in such a way that the maximum value of a distance from a customer to the closest facility is minimized -- we can use any IP formulation of the $k$-center problem as an alternative. Here we use the IP described in https://scipbook.readthedocs.io/en/latest/flp.html.

In [14]:
P = picos.Problem()

# Variables
z = picos.IntegerVariable("z", lower = 0)
a = picos.BinaryVariable("a", (n, n))
C = picos.BinaryVariable("C", n)

# Objective function
P.set_objective("min", z)
print(P)

# Constraints
for i in range(n):
  P += (sum([a[i,j] for j in range(n)]) == 1)
  
P += (sum([C[j] for j in range(n)]) <= k)

for i in range(n):
  for j in range(n):
    P += (a[i,j] <= C[j])

for i in range(n):
  P += (z >= sum([d[i,j] * a[i,j] for j in range(n)]))

P.solve(verbosity = 1, solver = 'gurobi')

Integer Linear Program
  minimize z
  over
    1×1 integer variable z (bounded below)
            PICOS 2.3.1            
Problem type: Integer Linear Program.
Searching a solution strategy.
Solution strategy:
  1. ExtraOptions
  2. GurobiSolver
Applying ExtraOptions.
Building a Gurobi problem instance.
Starting solution search.
Set parameter FeasibilityTol to value 1e-08
Set parameter OptimalityTol to value 1e-08
Set parameter BarQCPConvTol to value 1e-10
Set parameter MIPGapAbs to value 1e-06
-----------------------------------
         Gurobi Optimizer          
-----------------------------------
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 441 rows, 421 columns and 1620 nonzeros
Model fingerprint: 0xf48c4687
Variable types: 0 continuous, 421 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [

<feasible primal solution (claimed optimal) from gurobi>

In [6]:
print(f'Objective value: {P.value}')
print(f'Optimal solution')
print(f'z     = {z.np}')
print(f'a  = {a.np}')
print(f'C    = {C.np}')

Objective value: 56.0
Optimal solution
z     = 56.0
a  = [[-0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0.  1.]
 [ 0.  1.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0.]
 [-0.  0. -0.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0.
  -0. -0.]
 [-0.  1.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0.]
 [-0.  0.  0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0.  0.]
 [ 0.  0.  0.  0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0.]
 [ 0.  0.  0.  0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0.]
 [ 0.  0. -0.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0.
  -0. -0.]
 [-0. -0.  0. -0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0.
  -0. -0.]
 [ 0.  0.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0.
  -0. -0.]
 [ 0.  1.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0.]
 [-0.  0

The global optimum is also $z^{*}=56$ for this IP, but in this case we obtain a slightly different candidate subset, namely $C=\{2,5,14,17,20\}$.